# Описание проекта

Крупнейший транспортно-логистический холдинг построил сортировочный терминал для минеральных удобрений. В рамках автоматиизации рабочих процессов, возникла необходимость распознавания номеров автомобильного транспорта, контейнеров и железнодорожных вагонов - все они учавствуют в логистической цепочке.
В цепочке обычно учавствует один автомобиль, который перевозит два контейнера, куда засыпаются удобрения и один вагон из которого удобрения высыпаются.

Общие цели автоматизации состояли в следующем:
1) Отслеживание транспортных средств и контейнеров. Нужно точно следить за перемещением и идентификацией автомобилей, контейнеров и вагонов на протяжении всего пути;
2) Интеграция данных в единую систему. Система должна обеспечить получение данных о всех элементах логистической цепочки в реальном времени для оптимизации работы;
3) Управление загрузкой и разгрузкой. Чтобы точно отслеживать объемы и местоположение минеральных удобрений, важно иметь систему, которая будет связывать каждый контейнер с конкретным автомобилем и вагоном.

Итого необходимо было разработать три системы:
- Для распознавания номера контейнера;
- Для распознавания номера автомобиля;
- Для распознавания номера вагона.

# Данные

На начальном этапе реализации проекта данных не было, и пилотная версия системы разрабатывалась и обучалась на общедоступных изображениях, только после постройки терминала и монтирования камер, были собраны релевантные изображения и дообучены модели.

# Импорт библиотек и модулей

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from termcolor import cprint

import cv2
from ultralytics import YOLO

from paddleocr import PaddleOCR

# Разработка системы распознавания номеров

Объявим функцию для отображения изображений.

In [2]:
def show_image(path, title=None, width=None, figsize=(10, 10)):
    """
    Отображает изображение по заданному пути с возможностью добавления заголовка.

    Аргументы:
        path (str): Путь к изображению.
        title (str, optional): Заголовок для изображения. По умолчанию - None.

    Возвращает:
        None
    """
    # Загружаем изображение
    try:
        image = cv2.imread(path)
    except:
        image = path
    # Проверяем, удалось ли загрузить изображение
    if image is None:
        print(f"Ошибка: не удалось загрузить изображение по пути '{path}'")
        return
    # Преобразуем цвет из BGR в RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Задаем размер
    plt.figure(figsize=figsize)
    # Отображаем изображение
    plt.imshow(image)
    plt.axis('off')  # Скрываем оси для чистого отображения
    if title:
        plt.title(title)  # Добавляем заголовок, если передан
    plt.show()

## Распознавние номера контейнера

Номер контейнера состоит из 11 символов: 4 буквы и 7 цифр, причем последняя цифра является проверочнной - по ней можно проверить коректность номера. На данном объекте используются контейнеры с различными типами расположения номера, всего возможно три варианта:

1-ый вариант, это номер, у которого цифры и буквы расположены в один вертикальный ряд:

In [ ]:
path = os.path.join('parsed_images', 'number_v')
vert_numbers_list = os.listdir(path)
show_image(os.path.join('parsed_images', 'number_v', vert_numbers_list[0]), '1-ый тип номера')

2-ой вариант, это номер, у которого цифры и буквы расположены вертикально, но не в один ряд:

In [ ]:
path = os.path.join('parsed_images', 'number_v_dig')
sep_vert_numbers_list = os.listdir(path)
show_image(os.path.join(path, sep_vert_numbers_list[0]), '2-ой тип номера')

3-ой вариант, это номер, у которого цифры и буквы расположены в один горизонтальный ряд:

In [ ]:
path = os.path.join('parsed_images', 'number_h')
hor_numbers_list = os.listdir(path)
show_image(os.path.join(path, hor_numbers_list[0]), '3-ий тип номера')

Для распознавания текста были опробованы следующие библиотеки:
- PaddleOCR;
-  EasyOCR;
-   Tesseract;
-   apple_ocr. 

В итоге, в качестве модели для распознавания был выбран PaddleOCR. 

*Тут стоит уточнить, что на данном этапе не рассматривалась возможность дообучения моделей для детекции и распознавания текста, и PaddleOCR показал лучшие результаты на подготовленных изображениях.

### Общий папйплайн обработки номера

Для обработки номера контейнера используется следующий пайплайн:
1) Считывается изображение с камеры;
2) Изображение подается на вход YOLO модели, которая обучена для детекции различных типов номеров.
3) В зависимости от типа детектированного номера происходит дополнительная обработка номера;
4) Производится подготовка изображения номера;
5) Подготовленное изображение подается на вход OCR модели.

Теперь по порядку.

#### 1. Считывание изображений

Считывание, предобработка и вывод изображений производится с использование библиотеки **openCV**.

#### 2. Обучение модели и детектирование номеров

После считывания изоюражения необходимо детектировать номер. Для детекции номеров используется нейросетевая модель на архитектуре YOLO. Была выбрана маленькая модель (s) 11 поколения.

Модель обучалась с использованием пакета ultralytics. Меток детекции было 5: `['check_digit', 'number_h', 'number_v', 'number_v_dig', 'number_v_lett']`

С помощью сервиса RoboFlow было размечено 1556 изображений, после аугментации было получено 4588 изображений, которые использовались для обучения.

Обучение производилось в Google Colab с использованием GPU NVIDIA A100. 

Ниже представлен код обучения:

```python
if torch.backends.mps.is_available():
    device = 'mps'
elif torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'CPU'
    
optimizer = 'NAdam'
lr0 = 0.1
lrf = 0.001
epochs = 300
batch_size = 64
patience = 10
resume=False

model.train(data='/content/containers-15/data.yaml',
            project=saving_log_dir,
            resume=resume,
            device=device,
            lr0=lr0,
            lrf=lrf,
            cos_lr=True,
            optimizer=optimizer,
            epochs=epochs,
            batch=batch_size,
            patience=patience,
            val=True,
            visualize=True,
            verbose=True
)

Так выглядит один из тренировочных батчей:

In [ ]:
show_image(os.path.join('images_for_project', 'train_batch0_numbers.jpg'), 'Тренировочный батч')

Всего понадобилось 226 эпох при размере батча в 64 изображения. Обучение составило около 3 часов.

Ниже представлены метрики на лучшей эпохе:

| epoch | train/box_loss | train/cls_loss | train/dfl_loss | metrics/precision(B) | metrics/recall(B) | metrics/mAP50(B) | metrics/mAP50-95(B) 
|-------|----------------|----------------|----------------|----------------------|--------------------|------------------|---------------------|
| 223   | 0.69284       | 0.35574       | 0.84251       | 0.97653              | 0.96416           | 0.99117          | 0.86494             |

In [ ]:
figsize=(6, 6)
show_image(os.path.join('images_for_project', 'P_curve_numbers.png'), 'Кривая Precision', figsize=figsize)
show_image(os.path.join('images_for_project', 'R_curve_numbers.png'), 'Кривая Recall', figsize=figsize)
show_image(os.path.join('images_for_project', 'PR_curve_numbers.png'), 'Кривая Precision-Recall', figsize=figsize)
show_image(os.path.join('images_for_project', 'F1_curve_numbers.png'), 'Кривая F1', figsize=figsize)

Далее предсказание модели необходимо распарсить. Для этого объявим функцию, которая будет возвращать список пар `[тип номера, обрезанное_изображение]`.

In [8]:
number_searching_model = YOLO(os.path.join('models_for_search', 'new_number_searching_model.pt'))

def parse_predictions(image_path, show_cropped=False, show_predictions=False, model=None, conf=0.8, verbose=True):
    """
    Выполняет предсказание на изображении, обрезает найденные объекты и, при необходимости, отображает их.

    Аргументы:
        image_path (str или ndarray): Путь к изображению или само изображение.
        show_cropped (bool): Отображать ли каждое обрезанное изображение.
        show_predictions (bool): Показывать ли полные предсказания модели.
        model: Модель для предсказаний; если не передана, используется `number_searching_model`.
        conf (float): Порог уверенности для предсказаний.
        verbose (bool): Выводить ли сообщения.

    Возвращает:
        list: Список пар [класс, обрезанное_изображение] для найденных объектов или None, если объекты не обнаружены.
    """
    
    # Загружаем изображение
    try:
        image = cv2.imread(image_path)
    except:
        image = image_path  # если передано само изображение
    # Устанавливаем модель по умолчанию, если она не задана
    if model is None:
        model = number_searching_model
    # Выполняем предсказание
    cropped_images = []
    predictions = model.predict(image, show=False, verbose=verbose, conf=conf)
    # Показываем предсказания модели
    if show_predictions:
        predictions[0].show()
    # Обрабатываем предсказания
    classes = {0: 'check_digit', 1: 'number_h', 2: 'number_v', 3: 'number_v_dig', 4: 'number_v_lett'}
    for prediction in predictions:
        for box in prediction.boxes:
            x1, y1, x2, y2, _, cls = box.numpy().data[0]
            cropped_image = image[int(y1):int(y2), int(x1):int(x2)]
            cropped_images.append([classes.get(cls), cropped_image])
            # Отображаем обрезанное изображение, если нужно
            if show_cropped:
                show_image(cropped_image)
    # Выводим результат
    if len(cropped_images) == 0:
        cprint('No cargo number detected', 'red', 'on_black') if verbose else None
        return None
    else:
        cprint(f'Detected cargo number types is {[x[0] for x in cropped_images]}', 'green', 'on_black') if verbose else None
        return cropped_images

Пример использования функции для парсинга:

In [ ]:
image = os.path.join('parsed_images', 'number_v', vert_numbers_list[0])
parsed_vertical = parse_predictions(image, show_cropped=True)

In [ ]:
image = os.path.join('parsed_images', 'number_h', hor_numbers_list[0])
parsed_horizontal = parse_predictions(image, show_cropped=True)

In [ ]:
image = os.path.join('parsed_images', 'number_v_dig', sep_vert_numbers_list[0])
parsed_vetical_sep = parse_predictions(image, show_cropped=True)

#### 3. Обработка номера

После того как мы получили вырезанные номера, их нужно преобразовать. Горизонтальные номера (3 тип) не требуют преобразования - их после подготовки можно сразу подавать на вход OCR модели.

Номера у которых символы номера расположены в один вертикальный ряд (1 тип) преобразуем следующим образом:
1) С помощью другой YOLO модели найдем все символы на обрезанном изображении;
2) Вырежим эти символы, аналогично тому как мы это сделали с номерами;
3) 'Соберем' эти символы в один горизонтальный ряд.

Как и с случае с модели для детектирования номеров, мы будем использовать YOLO нейросетевую архитектуру. Изображения для обучения были получены с использованием модели для детектирования номеров - для 100 избражений были вырезанны номера и размечены символы, далее после аугментации получили 300 изображений и уже на них обучали нейросеть. Метка была одна: `symbol`

Пример тренировочного батча:

In [ ]:
show_image(os.path.join('images_for_project', 'train_batch0_symbols.jpg'), 'Тренировочный батч')

Всего понадобилось 78 эпох при размере батча в 32 изображения. Обучение составило около часа.

Ниже представлены метрики на лучшей эпохе:

| epoch | train/box_loss | train/cls_loss | train/dfl_loss | metrics/precision(B) | metrics/recall(B) | metrics/mAP50(B) | metrics/mAP50-95(B) 
|-------|----------------|----------------|----------------|----------------------|--------------------|------------------|---------------------|
| 78   | 0.51208      | 0.30328      | 0.81092       | 0.99894        | 0.9987        | 0.995     | 0.87016           |

In [ ]:
figsize=(6, 6)
show_image(os.path.join('images_for_project', 'P_curve_symbols.png'), 'Кривая Precision', figsize=figsize)
show_image(os.path.join('images_for_project', 'R_curve_symbols.png'), 'Кривая Recall', figsize=figsize)
show_image(os.path.join('images_for_project', 'PR_curve_symbols.png'), 'Кривая Precision-Recall', figsize=figsize)
show_image(os.path.join('images_for_project', 'F1_curve_symbols.png'), 'Кривая F1', figsize=figsize)

После детекции всех символов на обрезнанном изображении, также вырезаем эти символы и 'склеиваем' их в один горизонтальный номер.

Объявим функцию для преобразования номера к горизонтальному виду:

In [14]:
horizontal_model = YOLO(os.path.join('models_for_search', 'symbol_searching_model.pt'))

def get_horizontal_number(image, h=640, w=320, show_horizontal=False, show_original=False, conf=0.7, show_predicted=False, verbose=False):
    """
    Функция для поиска горизонтальных символов на изображении и отображения/обрезки этих символов.

    Аргументы:
        image (ndarray): Входное изображение для анализа.
        h (int): Высота изображения, в которое будет преобразовано каждое обрезанное изображение.
        w (int): Ширина изображения, в которое будет преобразовано каждое обрезанное изображение.
        show_horizontal (bool): Показывать ли итоговое изображение с горизонтальными символами.
        show_original (bool): Показывать ли оригинальное изображение.
        conf (float): Порог уверенности для предсказаний модели.
        show_predicted (bool): Показывать ли предсказания модели.
        verbose (bool): Выводить ли дополнительные сообщения в процессе выполнения.

    Возвращает:
        ndarray: Итоговое изображение с распознанными горизонтальными символами.
    """
    
    # Если нужно, показываем оригинальное изображение
    if show_original:
        cprint('Original image is:', 'green', 'on_black')
        show_image(image)
    # Получаем размеры изображения (высота и ширина)
    height, width, _ = image.shape
    # Если изображение вертикальное (высота больше ширины)
    if height > width:
        # Если включен параметр verbose, выводим сообщение о вертикальном изображении
        cprint('Vertical image is detected', 'green', 'on_black') if verbose else None
        # Прогнозируем символы на изображении с использованием модели
        predictions = horizontal_model.predict(image, save_crop=False, conf=conf, verbose=verbose)
        # Извлекаем найденные боксы (границы символов)
        boxes = predictions[0].boxes.numpy().data
        # Проверяем, что количество обнаруженных символов соответствует одному из допустимых вариантов
        if len(boxes) not in [11, 7, 4]:
            # Если количество символов неверное, выводим сообщение и возвращаем None
            cprint('Not enough symbols detected', 'red', 'on_black') if verbose else None
            return None
        # Создаем пустое изображение для дальнейшего объединения обрезанных символов
        res = np.zeros((h, 1, 3), np.uint8)
        # Сортируем боксы по вертикальной координате и обрезаем изображения символов
        for box in sorted(boxes, key=lambda x: x[1]):
            # Извлекаем координаты бокса
            x1, y1, x2, y2, conf, _ = box
            # Обрезаем изображение по этим координатам
            imgCrop = image[int(y1):int(y2), int(x1):int(x2)]
            # Изменяем размер обрезанного изображения и добавляем его в итоговое изображение
            res = cv2.hconcat([res, cv2.resize(imgCrop, (w, h))])
        # Если нужно, показываем предсказания модели
        if show_predicted:
            cprint('Predicted symbols is:', 'green', 'on_black')
            predictions[0].show()
    # Если изображение горизонтальное (ширина больше или равна высоте)
    else:
        # Если включен параметр verbose, выводим сообщение о горизонтальном изображении
        cprint('Horizontal image is detected', 'green', 'on_black') if verbose else None
        # В этом случае, просто возвращаем оригинальное изображение
        res = image
    # Если нужно, показываем итоговое изображение с горизонтальными символами
    if show_horizontal:
        cprint('Horizontal number is:', 'green', 'on_black')
        show_image(res)
    # Возвращаем итоговое изображение
    return res


In [ ]:
horizontal_number = get_horizontal_number(parsed_vertical[1][1], verbose=True, show_horizontal=True, show_original=True)

In [ ]:
horizontal_number = get_horizontal_number(parsed_horizontal[0][1], verbose=True, show_horizontal=True, show_original=True)

Номера, у которого цифры и буквы расположены вертикально, но не в один ряд (2 тип), обрабатываются аналогично, но добалвяется еще один шаг - полученные изображения с выхода первой модели для детекции с метками 'number_v_dig' и 'number_v_lett' 'склеиваем' сначала в один вертикальный номер, а затем детектируем на этом изображении символы и собираем в горизонатльный номер, как это было с номером 1-го типа.

________________________________________________________________________________________________________________________________________

In [26]:
def prepare_image(image, size=(640, 160), resize=False, gray=False, show=False, thresh=0, blure=15):
    """
    Подготавливает изображение для дальнейшей обработки: изменяет размер, переводит в нужный цветовой формат,
    применяет пороговую обработку и блюринг, если необходимо.

    Аргументы:
        image (ndarray): Входное изображение.
        size (tuple): Размер изображения для изменения.
        resize (bool): Флаг изменения размера.
        gray (bool): Флаг перевода изображения в оттенки серого.
        show (bool): Флаг отображения изображения после обработки.
        thresh (int): Порог для преобразования в бинарное изображение.
        blure (int): Уровень блюра для размытия.

    Возвращает:
        image (ndarray): Обработанное изображение.
    """
    # Меняем размер
    if resize:
        image = cv2.resize(image, size)
    
    # Преобразуем изображение в оттенки серого
    if gray or thresh:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Медианный блюр и пороговая обработка
    if thresh:
        image = cv2.medianBlur(image, blure)
        _, image = cv2.threshold(image, thresh, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Отображение
    if show:
        cprint('Prepared image is:', 'green', 'on_black')
        show_image(image)

    return image

In [24]:
det_model_path = os.path.join('models_for_search', 'det', 'en', 'en_PP-OCRv3_det_infer')
rec_model_path = os.path.join('models_for_search', 'rec', 'en', 'ch_ppocr_server_v2.0_rec_train')
cls_model_path = os.path.join('models_for_search', 'rec', 'en', 'ch_ppocr_mobile_v2.0_cls_infer')

ocr_reader = PaddleOCR(lang='en',
                       show_log=False,
                       use_angle_cls=True,
                       det_model_dir=det_model_path,
                       rec_model_dir=rec_model_path,
                       cls_model_dir=cls_model_path,
)

def get_text(file):
    """
    Распознает и возращает текст.

    Аргументы:
        file (ndarray): Входное изображение.

    Возвращает:
        text (str): Распознанный текст.
    """
    try:
        return ocr_reader.ocr(file)
    except:
        cprint('No text on the image', 'red', 'onblack')
        return None

In [19]:
def check_number(number, verbose=False):
    """
    Проверяет корректность контрольной цифры в номере на основе алгоритма вычисления контрольной суммы.
  
    Аргументы:
        number (str): Номер, в котором проверяется контрольная цифра.
        verbose (bool): Флаг для отображения информации о вычисленной контрольной цифре (по умолчанию False).

    Возвращает:
        bool: Возвращает True, если контрольная цифра правильная, и False, если нет.
    """

    alphabet = {**{'U': 32, 'M': 24, 'T': 31, 'L': 23}, **{str(x): x for x in range(10)}}
    res = 0
    # Проходим по всем символам в номере, кроме последнего, и рассчитываем контрольную сумму
    for i, sym in enumerate(number[:-1]):
        # Для каждого символа в номере прибавляем его значение с учетом его позиции
        res = res + 2**i * alphabet.get(sym, 0)
    res = res % 11 % 10
    cprint(f'Calculated check digit is: {res}', 'blue') if verbose else None
    return str(res) == number[-1]

In [20]:
def get_cargo_number(
    file,
    show_horizontal=False,
    show_original=False,
    show_prepared=False,
    print_pred_text=False,
    show_predictions=False,
    show_cropped=False,
    thresh_horizontal_number=0,
    thresh_vertical_number=1,
    thresh_check_digit=1,
    blure_horizontal_number=1,
    blure_vertical_number=7,
    blure_check_digit=7,
    size_number=(640, 160),
    size_check_digit=(200, 200),
    verbose=True,
):
    """
    Распознаёт номер контейнера на изображении и возвращает его в формате ISO 6346.

    Аргументы:
        file (str): Путь к входному изображению.
        show_horizontal (bool): Флаг отображения горизонтально обработанных изображений.
        show_original (bool): Флаг отображения исходного изображения.
        show_prepared (bool): Флаг отображения подготовленных изображений.
        print_pred_text (bool): Флаг отображения текста, полученного с OCR.
        show_predictions (bool): Флаг отображения предсказаний модели.
        show_cropped (bool): Флаг отображения обрезанных изображений.
        thresh_horizontal_number (int): Пороговая обработка для горизонтальных номеров.
        thresh_vertical_number (int): Пороговая обработка для вертикальных номеров.
        thresh_check_digit (int): Пороговая обработка для контрольной цифры.
        blure_horizontal_number (int): Уровень блюра для горизонтальных номеров.
        blure_vertical_number (int): Уровень блюра для вертикальных номеров.
        blure_check_digit (int): Уровень блюра для контрольной цифры.
        size_number (tuple): Размер изображения номера контейнера.
        size_check_digit (tuple): Размер изображения контрольной цифры.
        verbose (bool): Флаг подробного вывода сообщений.

    Возвращает:
        str: Распознанный номер контейнера, если он соответствует формату ISO 6346. Иначе None.
    """
    # Парсинг предсказаний модели и получение обрезанных изображений
    cropped_images = parse_predictions(
        file, show_cropped=show_cropped, show_predictions=show_predictions, verbose=verbose
    )
    if cropped_images is None:
        if verbose:
            cprint('No images to work', 'red', 'on_black')
        return None

    try:
        number_types = [x[0] for x in cropped_images]

        for image in cropped_images:
            # Обработка вертикальных номеров контейнеров
            if image[0] == 'number_v':
                number = get_horizontal_number(
                    image[1],
                    show_horizontal=show_horizontal,
                    show_original=show_original,
                    verbose=verbose,
                )
                prepared_image = prepare_image(
                    number,
                    thresh=thresh_vertical_number,
                    blure=blure_vertical_number,
                    show=show_prepared,
                    resize=True,
                    size=size_number,
                )
                text = get_text(prepared_image)[0][0][1][0]
                if print_pred_text:
                    cprint(f'Text after OCR is {text}', 'blue')

            # Обработка горизонтальных номеров контейнеров
            if image[0] == 'number_h':
                number = get_horizontal_number(
                    image[1],
                    show_horizontal=show_horizontal,
                    show_original=show_original,
                    verbose=verbose,
                )
                prepared_image = prepare_image(
                    number,
                    thresh=thresh_horizontal_number,
                    blure=blure_horizontal_number,
                    show=show_prepared,
                    resize=True,
                    size=size_number,
                )
                texts = get_text(prepared_image)
                if print_pred_text:
                    cprint(f'Text after OCR is {texts}', 'blue')

                # Обработка текстов и формирование итогового текста
                text_list = []
                for el in texts[0]:
                    if len(el[1][0]) in [4, 6]:
                        text_list.append(el[1][0])

                text = ''.join(sorted(text_list, reverse=True))
                if print_pred_text:
                    cprint(f'Text after preparing is {text}', 'blue')

            # Обработка контрольной цифры
            if image[0] == 'check_digit' and 'number_h' in number_types:
                prepared_image = prepare_image(
                    image[1],
                    thresh=thresh_check_digit,
                    blure=blure_check_digit,
                    show=show_prepared,
                    resize=True,
                    size=size_check_digit,
                )
                try:
                    check_digit_text = get_text(prepared_image)[0][0][1][0]
                    for el in check_digit_text:
                        if el.isdigit():
                            check_digit = el
                            break
                    if verbose:
                        cprint(f'Detected check digit is: {check_digit}', 'blue')
                except Exception:
                    check_digit = None
                    if verbose:
                        cprint('No check digit detected', 'red', 'on_black')

            # Обработка вертикальных частей номера (буквы и цифры)
            if image[0] == 'number_v_dig':
                number_v_dig = cv2.resize(image[1], (90, 600))

            if image[0] == 'number_v_lett':
                number_v_lett = cv2.resize(image[1], (90, 300))

        # Формирование полного вертикального номера
        if 'number_v_lett' in number_types and 'number_v_dig' in number_types:
            collected_number = cv2.vconcat([number_v_lett, number_v_dig])
            number = get_horizontal_number(
                collected_number,
                show_horizontal=show_horizontal,
                show_original=show_original,
                verbose=verbose,
            )
            prepared_image = prepare_image(
                number,
                thresh=1,
                blure=7,
                show=show_prepared,
                resize=True,
                size=(640, 160),
            )
            text = get_text(prepared_image)[0][0][1][0]

        # Постобработка текста для исправления формата
        if text[2] == 'T' and len(text) == 11:
            text = 'UMTU' + text[4:10] + text[-1]

        if text[2] == 'L' and len(text) == 11:
            text = 'UMLU' + text[4:10] + text[-1]

        if text[2] == 'L' and len(text) != 11 and check_digit:
            text = 'UMLU' + text[4:10] + check_digit

    except Exception as e:
        # Обработка ошибок
        cprint(e, 'red', 'on_black')
        return None

    if verbose:
        cprint(f'Detected text is {text}', 'blue')

    # Проверка корректности текста согласно стандарту ISO 6346
    if len(text) == 11 and text[0:4].isalpha() and text[4:].isdigit() and check_number(text, verbose=verbose):
        return text

    if verbose:
        cprint(f'Prepared text {text} is wrong', 'red', 'on_black')
    return None


In [ ]:
image = os.path.join('parsed_images', 'number_v', vert_numbers_list[0])
show_image(image)
get_cargo_number(image, show_original=True, show_cropped=False, show_prepared=True, show_horizontal=True, show_predictions=False)